<a href="https://colab.research.google.com/github/dooooing/2023-ESAA-/blob/main/0915_%EC%84%B8%EC%85%98_%EB%AA%A8%EB%8D%B8%ED%9B%88%EB%A0%A8_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10번
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___



확률적 경사 하강법, 미니배치 경사 하강법, 배치 경사 하강법

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

학습률이 너무 높음이 원인일 수 있음.

단, 훈련 에러 증가X 라면 모델이 훈련 세트에 과대적합했다고 결론

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

높은 편향이 문제

모델이 훈련 세트에 과소적합 가능. 규제하이퍼파라미터를 감소해야 함

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

**선형 < 릿지**

규제X < 규제O

**릿지 < 라쏘**

- 중요한 가중치 제외하고 0으로 바꾸기 가능
- 자동 특성 선택 효과

**라쏘 < 엘라스틱넷**

라소의 불규칙성 때문

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [7]:
from sklearn.datasets import load_iris
import numpy as np

In [8]:
iris = load_iris()

X = iris['data'][:, (2, 3)]
y = iris['target']

In [13]:
X_with_bias = np.c_[np.ones([len(X), 1]), X]
np.random.seed(2042)

In [14]:
test_ratio = 0.2
validation_ratio = 0.2
total_size = len(X_with_bias)

test_size = int(total_size * test_ratio)
validation_size = int(total_size * validation_ratio)
train_size = total_size - test_size - validation_size

In [17]:
rnd_indices = np.random.permutation(total_size)

X_train = X_with_bias[rnd_indices[:train_size]]
y_train = y[rnd_indices[:train_size]]

In [19]:
X_valid = X_with_bias[rnd_indices[train_size:-test_size]]
y_valid  = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [24]:
def to_one_hot(y):
  n_classes = y.max() + 1
  m = len(y)
  Y_one_hot = np.zeros((m, n_classes))
  Y_one_hot[np.arange(m), y] = 1
  return Y_one_hot

In [21]:
y_train[:10]

array([2, 0, 2, 1, 0, 2, 2, 1, 2, 0])

In [25]:
to_one_hot(y_train[:10])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [27]:
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot = to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [28]:
def softmax(logits):
  exps = np.exp(logits)
  exp_sums = np.sum(exps, axis=1, keepdims=True)
  return exps / exp_sums

In [29]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))

In [35]:
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha = 0.1
best_loss = np.infty

Theta = np.random.randn(n_inputs, n_outputs)

In [36]:
for iteration in range((n_iterations)):
  logits = X_train.dot(Theta)
  Y_proba = softmax(logits)
  loss = -np.mean(np.sum(Y_train_one_hot * np.log(Y_proba + epsilon), axis=1))
  error = Y_proba - Y_train_one_hot
  gradients = 1/m * X_train.T.dot(error) + np.r_[np.zeros([1, n_outputs]), alpha * Theta[1:]]
  Theta = Theta - eta * gradients

  logits = X_valid.dot(Theta)
  Y_proba = softmax(logits)
  xentropy_loss = -np.mean(np.sum(Y_valid_one_hot * np.log(Y_proba + epsilon), axis=1))
  l2_loss = 1/2 * np.sum(np.square(Theta[1:]))
  loss = xentropy_loss + alpha * l2_loss
  if iteration % 500 == 0:
    print(iteration, loss)
  if loss < best_loss:
    best_loss = loss
  else:
    print(iteration - 1, best_loss)
    print(iteration, loss, "early stopping!")
    break

0 1.7131861883987285
500 0.8184232091515297
1000 0.6954612647373447
1500 0.6325363013852694
2000 0.5964955583101414
2500 0.5734960811173818
3000 0.5574446591375319
3500 0.5454552807793422
4000 0.5360454126984733
4500 0.5283900872854292
5000 0.521995772721077


In [37]:
logits = X_valid.dot(Theta)
Y_proba = softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)

accuracy_score = np.mean(y_predict == y_valid)
accuracy_score

0.9666666666666667